In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from models import GNNClassifier
import torch
import numpy as np
import util
import scipy

In [9]:
random_state = 1
max_epoch = 100
batch_size = 100
max_dim = 40
batch_size = 64

model_config = {}
model_config["input_dim"] = max_dim
model_config["hidden_dim"] = 64
model_config["output_dim"] = 64
model_config["n_class"] = 2
model_config["c_u"] = 1
model_config["c_sigma"] = 2

torch.manual_seed(random_state)
# torch.cuda.manual_seed(random_state)
np.random.seed(random_state)

In [4]:
dataset_name = 'NCI1'
degree_as_tag = False
graphs, _  = util.load_data(dataset_name, degree_as_tag)
labels = np.array([g.label for g in graphs]).astype(int)

loading data
# classes: 2
# maximum node tag: 37
# data: 4110


In [24]:
graphs[0].neighbors

[[7],
 [7],
 [9],
 [9],
 [19],
 [6, 10],
 [5, 16],
 [0, 1, 11],
 [16, 17],
 [2, 3, 14],
 [5, 11, 12],
 [7, 10, 13],
 [10, 15],
 [11, 14],
 [9, 13, 15],
 [14, 12],
 [6, 8, 18],
 [19, 8],
 [16],
 [4, 17, 20],
 [19]]

In [5]:
help(graphs[0])

Help on S2VGraph in module util object:

class S2VGraph(builtins.object)
 |  S2VGraph(g, label, node_tags=None, node_features=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, g, label, node_tags=None, node_features=None)
 |      g: a networkx graph
 |      label: an integer graph label
 |      node_tags: a list of integer node tags
 |      node_features: a numpy float tensor, one-hot representation of the tag that is used as input to neural nets
 |      neighbors: list of neighbors (without self-loop)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [6]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


input_features = []
labels = []
for g in graphs:
    x = torch.tensor(g.node_features)
    x = F.pad(input=x, pad=(0, 0, 0, max_dim-x.shape[0]), mode='constant', value=0)
    input_features.append(x)
    labels.append(g.label)

input_features = torch.stack(input_features)
labels = torch.tensor(labels)
dataset = TensorDataset(input_features, labels)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.1, random_state=random_state)
print(len(train_dataset))
train_dataloader = DataLoader(train_dataset,batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size)

3699


In [38]:
!pip install torch_geometric

     |████████████████████████████████| 407 kB 736 kB/s eta 0:00:01
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=4d7fb2a84044fabc8009e4e0eddc04ae48e2e5780b9ce28fe116fe3f08fde116
  Stored in directory: /Users/k/Library/Caches/pip/wheels/ba/44/59/28a61b516dbc37ed93285752c61c2cf1724ba0a0dada6c1150
Successfully built torch-geometric


In [40]:
from torch_geometric.datasets import TUDataset
import os
path = os.path.join('./', 'data', 'TU')
dataset = TUDataset(path, name='MUTAG').shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]
test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=2)

ModuleNotFoundError: No module named 'torch_sparse'

In [ ]:
!pip install torch_sparse

     |████████████████████████████████| 48 kB 174 kB/s eta 0:00:011

In [7]:
from sklearn.metrics import accuracy_score

def train(model, optimizer, dataloader):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs = batch[0]
        targets = batch[1]
        optimizer.zero_grad()
        loss = model.get_loss(inputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 
    return


def test(model, dataloader):
    model.eval()
    total_acc = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0]
            targets = batch[1]
            predictions = model.predict(inputs, targets)
            acc = accuracy_score(predictions, targets)
            total_acc += acc
    
    total_acc /= len(dataloader)
            
    return total_acc

learning_rate = 0.001
model = GNNClassifier(model_config)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(1, epochs + 1):
    train_loss = train(model, optimizer, dataloader)
    test_acc = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, train_loss: {:.4f}, test_acc: {:.4f}'.format(epoch, trin_loss, test_acc))
    

NameError: name 'epochs' is not defined

In [33]:
import torch
inputs = torch.tensor([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
inputs = inputs.unsqueeze(0)
neighbors_index = [[0,1],[1,2],[2,1]]
neighbors_index = torch.tensor(neighbors_index)

In [26]:
inputs.shape

torch.Size([1, 3, 4])

In [34]:
torch.scatter_add(inputs,index = neighbors_index, dim=1)

TypeError: scatter_add() received an invalid combination of arguments - got (Tensor, dim=int, index=Tensor), but expected one of:
 * (Tensor input, int dim, Tensor index, Tensor src, *, Tensor out)
 * (Tensor input, name dim, Tensor index, Tensor src)


In [36]:
!pip install torch_scatter

  Using cached torch_scatter-2.0.9.tar.gz (21 kB)
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp39-cp39-macosx_10_9_x86_64.whl size=290315 sha256=6841d50cecac01cc27204c17d69cd171901504f3e0b83625d83ace1ab8d22899
  Stored in directory: /Users/k/Library/Caches/pip/wheels/0f/60/0a/78e62f3c2b8055590441b1cb389eb274e3740d8d7462c7abd8
Successfully built torch-scatter
